# Assemblies Quality Selection for NCBI Downloaded Assemblies

In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import re

## This script takes output list of biosamples from each species from the R script organize species

In [3]:
seq_path = "/Users/rx32940/Dropbox/5.Rachel-projects/Phylogeography/Lepto_assemblies_V2/ncbi_assemblies"

# get isolates in each species
species_table = pd.read_csv(seq_path + "/species_list/Leptospira interrogans", sep=",") 
species_acc = species_table['V1']

In [4]:

biosamples = pd.DataFrame()

biosamples = pd.read_csv(seq_path+"/multiqc_quast.txt", sep= "\t") # read in output of multiqc
biosamples['biosample_acc'] = biosamples["Sample"].str.split(" | ").str[0] # must split by " | ", or biosample str will have a space in the str, can't be merged in later steps
biosamples = biosamples.drop("Sample",1)


## For NCBI downloaded assemblies coverage extraction 

In [5]:
# get coverage for each isolate
path = "/Users/rx32940/Dropbox/5.Rachel-projects/Phylogeography/Lepto_assemblies_V2/ncbi_assemblies/"

# the xml file was fixed by addind a root and heading (one xml can only have one root)
doc = ET.parse(path + "ncbi_assemblies_full.xml") # xml object, biosample result file downloaded directly from ncbi with all_assemblies list

root = doc.getroot() # get the root element

full_cov = pd.DataFrame()
for isolate in root.findall("DocumentSummary"):
    acc = isolate.find("BioSampleAccn").text
    cov = isolate.find("Coverage").text
    org = isolate.find("Organism").text
    species = isolate.find("SpeciesName").text
    N50 = isolate.find("ScaffoldN50").text
    inst = isolate.find("SubmitterOrganization").text
    complete = isolate.find("AssemblyStatus").text
    proj = isolate.find("GB_BioProjects").find("Bioproj").find("BioprojectAccn").text
    
    coverage = pd.DataFrame({"biosample_acc":[acc],"coverage":[cov],"organism":[org],"species":[species], "scaffoldN50": [N50], "Institue":[inst], "Assembly_status":[complete],"bioproject":[proj]},
                            columns = ["biosample_acc","coverage","organism","species","scaffoldN50","Institue","Assembly_status","bioproject"])
    full_cov = full_cov.append(coverage)
    
# ncbi downloaded assemblies and their species name
species_dict = full_cov[["biosample_acc","species"]]

species_dict.to_csv(path + "ncbi_acc_species_dict.csv",header=None, index=False)

species_dict

pd.unique(species_dict["species"])
    


array(['Leptospira interrogans', 'Leptospira sp.',
       'Leptospira borgpetersenii', 'Leptospira biflexa',
       'Leptospira noguchii', 'Leptospira santarosai',
       'Leptospira weilii', 'Leptospira licerasiae',
       'Leptospira kirschneri', 'Leptospira inadai', 'Leptospira broomii',
       'Leptospira kmetyi', 'Leptospira alexanderi', 'Leptospira meyeri',
       'Leptospira wolffii', 'Leptospira fainei',
       'Leptospira mayottensis', 'Leptospira sp. Fiocruz LV3954',
       'Leptospira vanthielii', 'Leptospira yanagawae',
       'Leptospira terpstrae', 'Leptospira wolbachii',
       'Leptospira alstonii', 'Leptospira sp. Fiocruz LV4135',
       'Leptospira sp. P2653', 'Leptospira sp. B5-022',
       'Leptospira sp. serovar Kenya', 'Leptospira sp. ZV016',
       'Leptospira tipperaryensis', 'Leptonema illini',
       'Leptospira venezuelensis', 'Leptospira hartskeerlii',
       'Leptospira saintgironsiae', 'Leptospira adleri',
       'Leptospira perolatii', 'Leptospira baranto

In [6]:
full_cov
full_cov.to_csv(path+ "assemblies_coverage.csv") 
full_cov = pd.read_csv(path+"assemblies_coverage.csv").drop("Unnamed: 0",1)
full_cov["biosample_acc"][0]


'SAMN06434252'

In [7]:
biosamples = pd.merge(full_cov,biosamples)
# take only the isolates from the query species
# biosamples = biosamples[biosamples["biosample_acc"].index.isin(species_acc.index)]

biosamples
# biosamples["coverage"]


,biosample_acc,coverage,organism,species,scaffoldN50,Institue,Assembly_status,bioproject,L50,Duplication ratio,...,Misassembled contigs length,Reference length,# misassembled contigs,NGA75,NA75,LGA75,NA50,LGA50,LA75,LA50
0,SAMN06434252,30.0000,Leptospira interrogans serovar Canicola (bacte...,Leptospira interrogans,4218946,Universidade Federal de Pelotas,Chromosome,PRJNA376496,1.0,1.003,...,4570695.0,4698134.0,2.0,30352.0,32897.0,52.0,62085.0,24.0,49.0,23.0
1,SAMN06434273,30.0000,Leptospira interrogans serovar Canicola (bacte...,Leptospira interrogans,4208129,Universidade Federal de Pelotas,Chromosome,PRJNA376501,1.0,1.007,...,4560160.0,4698134.0,2.0,28968.0,30585.0,54.0,60422.0,24.0,51.0,23.0
2,SAMN07572185,15.4200,Leptospira sp. (bacteria),Leptospira sp.,65100,University of Illinois at Urbana-Champaign,Contig,PRJNA323575,15.0,1.000,...,0.0,4698134.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SAMN11389100,20.0000,Leptospira borgpetersenii (bacteria),Leptospira borgpetersenii,3580756,INDICASAT AIP,Chromosome,PRJNA543109,1.0,1.004,...,3887834.0,3931782.0,2.0,15022.0,15795.0,80.0,34042.0,37.0,78.0,37.0
4,SAMN11389098,20.0000,Leptospira borgpetersenii (bacteria),Leptospira borgpetersenii,3579904,INDICASAT AIP,Chromosome,PRJNA543109,1.0,1.004,...,3887110.0,3931782.0,2.0,14988.0,15170.0,81.0,34043.0,37.0,79.0,37.0
5,SAMN11389099,20.0000,Leptospira borgpetersenii (bacteria),Leptospira borgpetersenii,3580481,INDICASAT AIP,Chromosome,PRJNA543109,1.0,1.004,...,3887691.0,3931782.0,2.0,15022.0,15795.0,80.0,34043.0,37.0,78.0,37.0
6,SAMN11389084,20.0000,Leptospira borgpetersenii (bacteria),Leptospira borgpetersenii,3513910,INDICASAT AIP,Chromosome,PRJNA543109,1.0,1.004,...,3821116.0,3931782.0,2.0,14571.0,15170.0,83.0,34725.0,37.0,77.0,36.0
7,SAMN11389101,20.0000,Leptospira borgpetersenii (bacteria),Leptospira borgpetersenii,3581767,INDICASAT AIP,Chromosome,PRJNA543109,1.0,1.004,...,3888971.0,3931782.0,2.0,14988.0,15170.0,81.0,34043.0,37.0,79.0,37.0
8,SAMN11389090,20.0000,Leptospira interrogans (bacteria),Leptospira interrogans,4233607,INDICASAT AIP,Chromosome,PRJNA543109,1.0,1.007,...,4583691.0,4698134.0,2.0,60938.0,61586.0,26.0,132556.0,12.0,25.0,11.0
9,SAMN11389094,20.0000,Leptospira interrogans (bacteria),Leptospira interrogans,4213733,INDICASAT AIP,Chromosome,PRJNA543109,1.0,1.003,...,4563643.0,4698134.0,2.0,60640.0,61945.0,30.0,99422.0,14.0,28.0,13.0


In [9]:
biosamples_cov = biosamples[(biosamples["coverage"] == 0) | 
                            (biosamples["coverage"] >= 30) & 
                            (biosamples["N50"] > 10000) &
                           (biosamples["# contigs"] <= 500)][["biosample_acc","coverage","# contigs","Total length","N50","# misassemblies","# N's per 100 kbp","species","organism",'Institue',"Assembly_status","bioproject"]]

# len(biosamples_cov[biosamples_cov["coverage"] < 50])

biosamples_cov
# N50 = biosamples['N50'].describe()
# num_contigs = biosamples['# contigs'].describe()
# num_misasm = biosamples['# misassemblies'].describe()

# good_N50 = biosamples[biosamples.N50 > biosamples.N50.quantile(.90)]
# # good_length = biosamples[biosamples['Total length'] > biosamples['Total length'].quantile(.75)]
# # less_contigs = biosamples[biosamples["# contigs"] < biosamples["# contigs"].quantile(.10)]
# intersect = set(good_N50['biosample_acc']).intersection(set(less_contigs['biosample_acc']))

# picked = pd.DataFrame(intersect)
# print(good_N50)
# # N50.plot.kde()

,biosample_acc,coverage,# contigs,Total length,N50,# misassemblies,# N's per 100 kbp,species,organism,Institue,Assembly_status,bioproject
0,SAMN06434252,30.0000,2.0,4570695.0,4218946.0,128.0,207.89,Leptospira interrogans,Leptospira interrogans serovar Canicola (bacte...,Universidade Federal de Pelotas,Chromosome,PRJNA376496
1,SAMN06434273,30.0000,2.0,4560160.0,4208129.0,130.0,612.08,Leptospira interrogans,Leptospira interrogans serovar Canicola (bacte...,Universidade Federal de Pelotas,Chromosome,PRJNA376501
21,SAMN02603847,0.0000,2.0,4627366.0,4277185.0,98.0,0.00,Leptospira interrogans,Leptospira interrogans serovar Copenhageni str...,Sao Paulo state (Brazil) Consortium,Complete Genome,PRJNA10687
22,SAMN02603615,0.0000,2.0,3931782.0,3614446.0,0.0,0.00,Leptospira borgpetersenii,Leptospira borgpetersenii serovar Hardjo-bovis...,Monash University,Complete Genome,PRJNA16146
23,SAMN02603616,0.0000,2.0,3876235.0,3576473.0,0.0,0.00,Leptospira borgpetersenii,Leptospira borgpetersenii serovar Hardjo-bovis...,Monash University,Complete Genome,PRJNA16148
24,SAMN02603617,0.0000,3.0,3956089.0,3603977.0,0.0,0.00,Leptospira biflexa,Leptospira biflexa serovar Patoc strain 'Patoc...,Monash University,Complete Genome,PRJNA16153
25,SAMN02603347,0.0000,3.0,3951448.0,3599677.0,0.0,0.00,Leptospira biflexa,Leptospira biflexa serovar Patoc strain 'Patoc...,Institut Pasteur,Complete Genome,PRJNA20133
26,SAMN02603127,0.0000,2.0,4698134.0,4338762.0,0.0,0.00,Leptospira interrogans,Leptospira interrogans serovar Lai str. 56601 ...,"Chinese National HGC, Shanghai",Complete Genome,PRJNA293
34,SAMN00255265,34.4000,16.0,4705262.0,617754.0,96.0,0.00,Leptospira interrogans,Leptospira interrogans serovar Canicola str. L...,J. Craig Venter Institute,Contig,PRJNA65063
35,SAMN02436336,51.0000,229.0,4679663.0,37187.0,85.0,0.00,Leptospira interrogans,Leptospira interrogans serovar Djasiman str. L...,J. Craig Venter Institute,Contig,PRJNA65061


In [15]:
biosamples_cov.to_csv(seq_path +"/ncbi_assemblies_filtered.csv") # write all into a file